# Was sind die größten und komplexesten Methoden?

## 1. Verbindung zur Datenbank
Es wird eine Verbindung zur Neo4j-Datenbank aufgebaut.

In [3]:
import py2neo

graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password='neo4j')

## 2. Cypher-Abfrage
Es wird eine Abfrage an die Datenbank gestellt. Das Ergebnis wird in einem Dataframe (pandas) gespeichert.

In [4]:
import pandas as pd

query ="MATCH (t:Type)-[:HAS_SOURCE]->(f), (t)-[:DECLARES]->(m:Method) RETURN t.fqn as type, m.signature as signature,  sum(m.effectiveLineCount) as loc, sum(m.cyclomaticComplexity) as complexity"
df = pd.DataFrame(graph.run(query).data())


## 3. Datenaufbereitung
Zur Kontrolle werden die ersten fünf Zeilen des Ergebnisses der Abfrage als Tabelle ausgegeben.

In [5]:
df.head()

,complexity,loc,signature,type
0,1,2,void <init>(),org.junit.rules.TestWatchmanTest$FailingTest
1,2,2,java.lang.Class getType(),org.junit.experimental.theories.ParameterSigna...
2,1,2,void forgetMax(),org.junit.tests.experimental.max.JUnit38Sortin...
3,0,0,void run(junit.framework.TestResult),junit.tests.extensions.ExtensionTest$TornDown
4,1,6,void doesNotConsiderParametersWhichChangedAfte...,org.junit.runners.parameterized.TestWithParame...


Im folgenden Codeabschnitt wird das Dataframe absteigend entsprechend der Anzahl der Quelltextzeilen (loc) sortiert und die ersten zehn Methoden extrahiert. Die Werte werden in den Variablen <font face="Courier">x_loc</font> und <font face="Courier">y_method_loc</font> gespeichert.

In [18]:
a=df.sort_values('loc',ascending=False)
b=a.head(10)
def x_loc(x):
    x=b['loc'].tolist()
    return x
def y_method_loc(y):
    liste=[]
    y=b['signature'].tolist()
    for i in y:
        g=i.split(' ')[-1]
        liste.append(g)
    return liste
print(b)
print(y_method_loc(b))

      complexity  loc                                          signature  \
3901           3   78                                       void setUp()   
5118          14   68     junit.framework.Test getTest(java.lang.String)   
304            1   60       java.lang.Object[][] testsWithEventMatcher()   
2388          63   56                          void testMoneyBagEquals()   
2763           1   55       java.util.Collection testsWithEventMatcher()   
1208          26   50  junit.framework.TestResult start(java.lang.Str...   
3825          35   49                             void testMoneyEquals()   
4428           7   49                                       void setUp()   
23            14   49                               void testElementAt()   
1934           7   42                                       void setUp()   

                                       type  
3901     junit.tests.runner.StackFilterTest  
5118            junit.runner.BaseTestRunner  
304      org.junit.rules.

Im folgenden Codeabschnitt wird das Dataframe absteigend entsprechend der Komplexität (complexity) sortiert und die ersten zehn Methoden extrahiert. Die Werte werden in den Variablen <font face="Courier">x_complexity</font> und <font face="Courier">y_method_complexity</font> gespeichert.

In [20]:
a=df.sort_values('complexity',ascending=False)
c=a.head(10)

def x_complexity(x):
    x=c['complexity'].tolist()
    return(x)

def y_method_complexity(y):
    liste=[]
    y=c['signature'].tolist()
    for i in y:
        g=i.split(' ')[-1]
        liste.append(g)
    return liste



print(x_complexity(c))
print(y_method_complexity(c))
print(c)

[63, 36, 35, 35, 27, 26, 24, 24, 22, 21]
['testMoneyBagEquals()', 'execTest(java.lang.String,boolean)', 'testMoneyEquals()', 'testCapacity()', 'testMoneyBagEquals()', 'start(java.lang.String[])', 'equals(java.lang.Object)', 'readPreferences()', 'processArguments(java.lang.String[])', 'verifySuccess(junit.framework.TestCase)']
      complexity  loc                                          signature  \
2388          63   56                          void testMoneyBagEquals()   
4684          36   36            void execTest(java.lang.String,boolean)   
3825          35   49                             void testMoneyEquals()   
581           35   35                                void testCapacity()   
4670          27   24                          void testMoneyBagEquals()   
1208          26   50  junit.framework.TestResult start(java.lang.Str...   
4964          24   36                   boolean equals(java.lang.Object)   
5394          24   24                             void readPrefe

## 4. Visualisierung
Die Daten werden mittels eines Bar Charts visualisiert (https://plot.ly/python/horizontal-bar-charts/, x=x_loc, y=y_method_loc).

In [16]:
import plotly.plotly as py
import plotly.graph_objs as go

data = [go.Bar(
            x=x_loc(b),
            y=y_method_loc(b),
            orientation = 'h'
)]

py.iplot(data, filename='größte Methoden')

Die Daten werden mittels eines Bar Charts visualisiert (https://plot.ly/python/horizontal-bar-charts/, x=x_complexity, y=y_method_complexity).

In [21]:
import plotly.plotly as py
import plotly.graph_objs as go

data = [go.Bar(
            x=x_complexity(c),
            y=y_method_complexity(c),
            orientation = 'h'
)]

py.iplot(data, filename='komplexeste Methoden')